## **📌 Tenserflow로 실습하는 CNN**

###  1. 기본 설정 ❗

> 분석에 필요한 라이브러리



In [ ]:
import os 
import numpy as np 
import pandas as pd #csv 파일 I/O
import matplotlib.pyplot as plt #시각화

In [ ]:
import tensorflow as tf #딥러닝 프레임워크
from tensorflow.keras import layers
from tensorflow.keras import Model
from os import getcwd
import cv2

## 2. 데이터 불러오기 📂



> 데이터 불러오기



In [ ]:
# 구글 드라이브 > MyDrive에 폴더 생성 후 데이터 직접 업로드
# > 코랩 파일에 들어와서 경로 복사 후 train_dir, test_dir에 할당
train_dir='/content/drive/MyDrive/zin/seg_train/seg_train'
test_dir='/content/drive/MyDrive/zin/seg_test/seg_test'

## 3. 이미지 데이터 전처리 

> target이 분류되어 있는 파일을 위한 label encoding



In [ ]:
# 분석을 위한 이미지 크기 및 배치 사이즈 설정

IMG_SIZE = (150,150)
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    directory=train_dir,
    image_size=IMG_SIZE,
    label_mode = "categorical", #label 형태 설정
    batch_size=64)

test_data = tf.keras.preprocessing.image_dataset_from_directory(
    directory=test_dir,
    image_size=IMG_SIZE,
    label_mode = "categorical", #label 형태 설정
    batch_size=64)

Found 14034 files belonging to 6 classes.
Found 3000 files belonging to 6 classes.


In [ ]:
# 타겟변수 확인
train_data.class_names

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']

In [ ]:
# 타겟 범주 :class_names x 
class_names = ['mountain', 'street', 'glacier', 'buildings', 'sea', 'forest']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}
nb_classes = len(class_names)

In [ ]:
# 파일별로 분류된 이미지 라벨링 함수
def load_data():
    train_test = ['seg_train/seg_train', 'seg_test/seg_test']
    size = (150, 150)
    output = [] 
    
    for dataset in train_test:
        directory = "/content/drive/MyDrive/zin/" + dataset
        images = []
        labels = []
        print("Loading {}".format(dataset))

        for folder in os.listdir(directory):
            label = class_names_label[folder]
            for file in os.listdir(directory + "/" + folder):
                img_path = directory + "/" + folder + "/" + file
                curr_img = cv2.imread(img_path)
                curr_img = cv2.resize(curr_img, size)
                images.append(curr_img)
                labels.append(label)
        images = np.array(images, dtype='float32')
        labels = np.array(labels, dtype='int32')

        images = images / 255.0 #데이터 정규화

        output.append((images, labels))

    return output

n_classes = 6

In [ ]:
(train_images, train_labels),(test_images, test_labels) = load_data()

Loading seg_train/seg_train
Loading seg_test/seg_test


In [ ]:
# train_image 형태
train_images.shape

NameError: ignored

In [ ]:
# test_image 형태
test_images.shape

In [ ]:
# 데이터 셋 중 하나의 이미지 확인
IMG_INDEX = 1126

plt.imshow(train_images[IMG_INDEX] ,cmap=plt.cm.binary)
plt.xlabel(class_names[train_labels[IMG_INDEX]])
plt.show()

NameError: ignored

# 3. CNN 실습 

> CNN 모델링



In [ ]:
# 모델링
model_0 = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255), # 데이터 정규화
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=IMG_SIZE+(3,)),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='softmax')
])

In [ ]:
# 모델 compile
model_0.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])




> 모델 예측



In [ ]:
# 예측
history_0 = model_0.fit(train_data,
                        epochs=10,
                        steps_per_epoch=len(train_data),
                        validation_data=test_data,
                        validation_steps=len(test_data))

Epoch 1/10
 66/220 [========>.....................] - ETA: 24:12 - loss: 1.3235 - accuracy: 0.4659

KeyboardInterrupt: ignored

In [ ]:
# 모델 요약 확인
model_0.summary()

# 4. CNN 모델 평가 

> 그래프를 통해 loss 값 확인

In [ ]:
def plot_loss_curves(history):
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Epoch에 따른 손실 정도 표로 나타내기
  plt.plot(epochs, loss, label='training_loss') #train loss
  plt.plot(epochs, val_loss, label='val_loss') #validation loss
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Epoch에 따른 정확도 표로 나타내기
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();
    

plot_loss_curves(history_0)

In [ ]:
# Let's evaluate the model
model_0.evaluate(test_data)

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='network_regression_model_shapes.png', show_shapes=True)